# Running NBody scan on K16, and saving times and durations

In [10]:
#can we find K16 while assuming basic values for the e and i parameters?
import copy
import numpy as np
import matplotlib.pyplot as p
import astropy.units as u
import funcs
import pickle

%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#Load lightcurve

lc = np.genfromtxt('Lightcurves/K16_clean_noecl.txt')
time = lc[:,0]+2454833
flux = lc[:,1]
err = lc[:,2]

In [3]:
# Kepler 16
cb = funcs.CBSystem(m1 = 0.6897,f1 = 1.,m2 = 0.20255,f2 = 0.01555,
                    ab = 0.22431,r1 = 0.6489 * u.Rsun.to('au'),r2 = 0.22623 * u.Rsun.to('au'),
                    eb = 0.15944,ib = np.deg2rad(90.3401),wb = np.deg2rad(263.464),
                    fb = np.deg2rad(186.53239),
                    mp = .03e-3,ap = 0.7048,rp = 0.7538 * u.Rjupiter.to('au'),ep = 0.0069,
                    ip = np.deg2rad(90.0322),wp = np.deg2rad(318.0),fp = np.deg2rad(148.92),
                    Wp = np.deg2rad(0.003),
                    t0 = 2455212.12316)
ab0 = cb.ab
mb0 = cb.m1 + cb.m2
p_p0 = (cb.ap**3/(cb.m1+cb.m2))**(1./2.)*365.25 #in days
fp0 = np.deg2rad(148.92)
print(1/np.sqrt( (cb.m1+cb.m2)/cb.ab**3 )*365.25)
print(p_p0)

timing_precision = 30./86400./365.25 * (2*np.pi)  #in years/2pi
dt = 0.02
#t = np.arange(2454953,2456424,dt)
#t = np.arange(np.min(time),np.max(time),dt)
t = time

41.0790034492
228.79461019691053


In [4]:
tts = funcs.reb_cb(cb,tmin=np.min(t),tmax=np.max(t))
tts_2,tds_2 = funcs.reb_cb_dvm(cb,0,2,tmin=np.min(t),tmax=np.max(t), timing_precision=timing_precision)
    
ok = tts[:,0] == 20
#ref_tts = tts[ok]
ref_tts = tts_2[tds_2>0]
ref_tds = tds_2[tds_2>0]
print(ref_tts)
print(ref_tds)

[ 2454973.42262594  2455203.69855133  2455425.21181695  2455655.48738862
  2455876.9974707   2456107.27304238  2456328.77339701]
[ 0.29092152  0.29507898  0.29054443  0.2937785   0.28803247  0.28898602
  0.28222904]


In [5]:
#degrading our knowledge (assume we know eb and wb):
#for now, radii left as we know them - this will change transit durations, and whether some marginal transits happen.

cb.ib = np.pi/2.
cb.ip = np.pi/2.
cb.Wp = 0.0
cb.ep = 0.0
#cb.wp left as is because it defines f_p
cb.mp = 0.0

In [7]:

#define range over which to scan
ppset = np.linspace(p_p0*0.97,p_p0*1.03,101)
#fpset = np.linspace(cb.fp*0.98,cb.fp*1.02,11)
print(np.diff(1./ppset))
nbins = (1/200. - 1/500.) / (2.5e-6)
print(nbins)
invppset = np.linspace(1/500.,1/200.,nbins)
ppset = 1./invppset[::-1] #uniform in frequency
fpset = np.linspace(0,2*np.pi,700)
print(ppset)
print(fpset)

[ -2.78543764e-06  -2.78199599e-06  -2.77856072e-06  -2.77513181e-06
  -2.77170925e-06  -2.76829301e-06  -2.76488308e-06  -2.76147945e-06
  -2.75808210e-06  -2.75469101e-06  -2.75130618e-06  -2.74792758e-06
  -2.74455520e-06  -2.74118902e-06  -2.73782904e-06  -2.73447522e-06
  -2.73112757e-06  -2.72778606e-06  -2.72445068e-06  -2.72112141e-06
  -2.71779824e-06  -2.71448116e-06  -2.71117014e-06  -2.70786518e-06
  -2.70456626e-06  -2.70127336e-06  -2.69798647e-06  -2.69470558e-06
  -2.69143067e-06  -2.68816173e-06  -2.68489873e-06  -2.68164168e-06
  -2.67839055e-06  -2.67514533e-06  -2.67190600e-06  -2.66867255e-06
  -2.66544497e-06  -2.66222324e-06  -2.65900734e-06  -2.65579727e-06
  -2.65259302e-06  -2.64939455e-06  -2.64620187e-06  -2.64301495e-06
  -2.63983379e-06  -2.63665837e-06  -2.63348867e-06  -2.63032469e-06
  -2.62716641e-06  -2.62401381e-06  -2.62086688e-06  -2.61772561e-06
  -2.61458998e-06  -2.61145999e-06  -2.60833561e-06  -2.60521683e-06
  -2.60210365e-06  -2.59899604e-06

/Users/davidarmstrong/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


In [8]:
#run scan, querying statistic at each point
tts_all = {}
tds_all = {}

#horrible for loop 

for ipp,pp in enumerate(ppset):
    cb.ap = ( (cb.m1+cb.m2) * (pp/365.25)**2 )**(1./3.)
    if ipp%5 == 0:
        print(ipp/len(ppset))
    tts_all[str(pp)[:6]] = {}
    tds_all[str(pp)[:6]] = {}
       
    for fp in fpset:
        cb.fp = fp
                     
        ttimes, tdurs = funcs.reb_cb_dvm(cb, 0, 2, tmin=np.min(time), 
                                        tmax=np.max(time), timing_precision=timing_precision)
                            
        tts_all[str(pp)[:6]][str(fp)[:6]] = ttimes[tdurs>0]
        tds_all[str(pp)[:6]][str(fp)[:6]] = tdurs[tdurs>0]

0.0
0.004166666666666667
0.008333333333333333
0.0125
0.016666666666666666
0.020833333333333332
0.025
0.029166666666666667
0.03333333333333333
0.0375
0.041666666666666664
0.04583333333333333
0.05
0.05416666666666667
0.058333333333333334
0.0625
0.06666666666666667
0.07083333333333333
0.075
0.07916666666666666
0.08333333333333333
0.0875
0.09166666666666666
0.09583333333333334
0.1
0.10416666666666667
0.10833333333333334
0.1125
0.11666666666666667
0.12083333333333333
0.125
0.12916666666666668
0.13333333333333333
0.1375
0.14166666666666666
0.14583333333333334
0.15
0.15416666666666667
0.15833333333333333
0.1625
0.16666666666666666
0.17083333333333334
0.175
0.17916666666666667
0.18333333333333332
0.1875
0.19166666666666668
0.19583333333333333
0.2
0.20416666666666666
0.20833333333333334
0.2125
0.21666666666666667
0.22083333333333333
0.225
0.22916666666666666
0.23333333333333334
0.2375
0.24166666666666667
0.24583333333333332
0.25
0.25416666666666665
0.25833333333333336
0.2625
0.26666666666666666

In [12]:
with open('K16_tts_all.pickle', 'wb') as handle:
    pickle.dump(tts_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

#67.8Mb - could be worse!
with open('K16_tds_all.pickle', 'wb') as handle:
    pickle.dump(tds_all, handle, protocol=pickle.HIGHEST_PROTOCOL)
